# Applying LSTM Models on Raw Data

In [1]:
# Importing Libraries

import pandas as pd
import numpy as np

# Import Keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout
from keras.layers import BatchNormalization
from keras.regularizers import L1L2

In [2]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

# function to print the confusion matrix

def confusion_matrix(Y_true, Y_pred):
    
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

    
    result = confusion_matrix(Y_true, Y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(result, 
                xticklabels= list(ACTIVITIES.values()), 
                yticklabels=list(ACTIVITIES.values()), 
                annot=True, fmt="d");
    plt.title("Confusion matrix")
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show() 

### Loading Data

In [4]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
    ]

In [5]:
# define a function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [6]:
def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'/content/drive/MyDrive/UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).to_numpy()

In [7]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [9]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [10]:
#function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [11]:
# Loading the train and test data
X_train, X_test, y_train, y_test = load_data()

In [12]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


 1. Defining the Architecture of 1-Layer of LSTM

In [13]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                5376      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 6)                 198       
                                                                 
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [15]:
# Training the model
model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test),epochs=epochs)

Epoch 1/30
460/460 [==============================] - 12s 10ms/step - loss: 1.1997 - accuracy: 0.5044 - val_loss: 0.9316 - val_accuracy: 0.5948
Epoch 2/30
460/460 [==============================] - 3s 8ms/step - loss: 0.7876 - accuracy: 0.6547 - val_loss: 0.7006 - val_accuracy: 0.7133
Epoch 3/30
460/460 [==============================] - 5s 10ms/step - loss: 0.6394 - accuracy: 0.7424 - val_loss: 0.6620 - val_accuracy: 0.7458
Epoch 4/30
460/460 [==============================] - 3s 8ms/step - loss: 0.5730 - accuracy: 0.7707 - val_loss: 0.7624 - val_accuracy: 0.7027
Epoch 5/30
460/460 [==============================] - 4s 9ms/step - loss: 0.4698 - accuracy: 0.8392 - val_loss: 0.7096 - val_accuracy: 0.7781
Epoch 6/30
460/460 [==============================] - 5s 11ms/step - loss: 0.3910 - accuracy: 0.8758 - val_loss: 0.5197 - val_accuracy: 0.8266
Epoch 7/30
460/460 [==============================] - 4s 8ms/step - loss: 0.3167 - accuracy: 0.8977 - val_loss: 0.4726 - val_accuracy: 0.8527
Ep

In [16]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 4ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,510,0,0,0,0,27
SITTING,0,390,97,0,1,3
STANDING,0,64,466,0,2,0
WALKING,0,0,0,439,51,6
WALKING_DOWNSTAIRS,0,1,0,4,414,1
WALKING_UPSTAIRS,0,1,2,26,13,429


In [17]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 0s 4ms/step - loss: 0.6049 - accuracy: 0.8985

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.6048541069030762, 0.8985409140586853]


   2. Defining the Architecture of 2-Layer of LSTM with more hyperparameter tunning

#### 2.1 First Model for 2-Layer of LSTM with more hyperparameter tunning

In [18]:
# Initializing parameters
n_epochs = 30
n_batch = 16
n_classes = _count_classes(y_train)

# Bias regularizer value - we will use elasticnet
reg = L1L2(0.01, 0.01)

In [19]:
# Model execution
model = Sequential()
model.add(LSTM(48, input_shape=(timesteps, input_dim), return_sequences=True,bias_regularizer=reg ))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(32))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 128, 48)           11136     
                                                                 
 batch_normalization (BatchN  (None, 128, 48)          192       
 ormalization)                                                   
                                                                 
 dropout_1 (Dropout)         (None, 128, 48)           0         
                                                                 
 lstm_2 (LSTM)               (None, 32)                10368     
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 198       
                                      

In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 12s 15ms/step - loss: 1.5419 - accuracy: 0.6927 - val_loss: 1.0551 - val_accuracy: 0.7981
Epoch 2/30
460/460 [==============================] - 7s 15ms/step - loss: 0.7324 - accuracy: 0.8751 - val_loss: 0.5578 - val_accuracy: 0.8911
Epoch 3/30
460/460 [==============================] - 6s 13ms/step - loss: 0.4112 - accuracy: 0.9123 - val_loss: 0.5101 - val_accuracy: 0.8517
Epoch 4/30
460/460 [==============================] - 8s 18ms/step - loss: 0.2974 - accuracy: 0.9142 - val_loss: 0.3355 - val_accuracy: 0.9023
Epoch 5/30
460/460 [==============================] - 6s 14ms/step - loss: 0.2495 - accuracy: 0.9189 - val_loss: 0.3902 - val_accuracy: 0.8802
Epoch 6/30
460/460 [==============================] - 7s 16ms/step - loss: 0.1918 - accuracy: 0.9317 - val_loss: 0.3483 - val_accuracy: 0.9023
Epoch 7/30
460/460 [==============================] - 6s 14ms/step - loss: 0.1725 - accuracy: 0.9378 - val_loss: 0.3657 - val_accuracy: 0.895

In [22]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 1s 5ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,536,1,0,0,0,0
SITTING,0,419,68,0,0,4
STANDING,0,115,416,1,0,0
WALKING,0,0,0,470,26,0
WALKING_DOWNSTAIRS,0,0,0,0,416,4
WALKING_UPSTAIRS,0,4,0,6,9,452


In [23]:
score = model.evaluate(X_test, y_test)

print("\n   categorica_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 6ms/step - loss: 0.2768 - accuracy: 0.9192

   categorica_crossentropy  ||   accuracy 
  ____________________________________
[0.2768016457557678, 0.91923987865448]


#### 2.2 Second Model for 2-Layer of LSTM with more hyperparameter tunning

In [24]:
# Model execution
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, input_dim), return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.50))
model.add(LSTM(48))
model.add(Dropout(0.50))
model.add(Dense(n_classes, activation='sigmoid'))
print("Model Summary: ")
model.summary()

Model Summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 128, 64)           18944     
                                                                 
 batch_normalization_1 (Batc  (None, 128, 64)          256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 128, 64)           0         
                                                                 
 lstm_4 (LSTM)               (None, 48)                21696     
                                                                 
 dropout_4 (Dropout)         (None, 48)                0         
                                                                 
 dense_2 (Dense)             (None, 6)                 294       
                                      

In [25]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
# Training the model
model.fit(X_train, y_train, batch_size=n_batch, validation_data=(X_test, y_test), epochs=n_epochs)

Epoch 1/30
460/460 [==============================] - 12s 16ms/step - loss: 1.6279 - accuracy: 0.7284 - val_loss: 1.0316 - val_accuracy: 0.8633
Epoch 2/30
460/460 [==============================] - 7s 16ms/step - loss: 0.7092 - accuracy: 0.8984 - val_loss: 0.4963 - val_accuracy: 0.8941
Epoch 3/30
460/460 [==============================] - 6s 13ms/step - loss: 0.3547 - accuracy: 0.9161 - val_loss: 0.3265 - val_accuracy: 0.8853
Epoch 4/30
460/460 [==============================] - 7s 16ms/step - loss: 0.2086 - accuracy: 0.9293 - val_loss: 0.2557 - val_accuracy: 0.9050
Epoch 5/30
460/460 [==============================] - 6s 13ms/step - loss: 0.2707 - accuracy: 0.9081 - val_loss: 0.3233 - val_accuracy: 0.8907
Epoch 6/30
460/460 [==============================] - 7s 16ms/step - loss: 0.1797 - accuracy: 0.9336 - val_loss: 0.2454 - val_accuracy: 0.9121
Epoch 7/30
460/460 [==============================] - 6s 14ms/step - loss: 0.1508 - accuracy: 0.9456 - val_loss: 0.2797 - val_accuracy: 0.910

In [27]:
# Confusion Matrix
confusion_matrix(y_test, model.predict(X_test))

93/93 [==============================] - 2s 6ms/step


Pred,LAYING,SITTING,STANDING,WALKING,WALKING_DOWNSTAIRS,WALKING_UPSTAIRS
True,,,,,,
LAYING,537,0,0,0,0,0
SITTING,3,383,102,0,0,3
STANDING,0,93,439,0,0,0
WALKING,0,0,0,471,15,10
WALKING_DOWNSTAIRS,0,0,0,1,415,4
WALKING_UPSTAIRS,0,0,0,3,24,444


In [28]:
score = model.evaluate(X_test, y_test)

print("\n   categorical_crossentropy  ||   accuracy ")
print("  ____________________________________")
print(score)

93/93 [==============================] - 1s 6ms/step - loss: 0.3191 - accuracy: 0.9125

   categorical_crossentropy  ||   accuracy 
  ____________________________________
[0.3190773129463196, 0.9124533534049988]
